## Dependencies 

In [1]:
# Import dependencies
from sklearn import model_selection
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error
import seaborn as sns
from math import sqrt

from yellowbrick.regressor import residuals_plot, prediction_error 
from scipy.stats import randint as sp_randint



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from config import password, g_key
import gmaps


In [2]:
# Database dependencies
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

# Database credentials
DATABASES = {
    'finalproject':{
        'POSTGRES_ADDRESS' :'finalproject.c969jxlkzz2v.us-east-2.rds.amazonaws.com',
        'POSTGRES_PORT' : '5432',
        'POSTGRES_USERNAME' : "postgres",
        'POSTGRES_PASSWORD' : password,
        'POSTGRES_DBNAME' : 'housingdata',
    },
}

# Choose the database to use
db = DATABASES['finalproject']

# Connect to postgres
postgres_str = ('postgresql+psycopg2://{username}:{password}@{ipaddress}:{port}/{dbname}'
               .format(username= db['POSTGRES_USERNAME'],
                      password= db['POSTGRES_PASSWORD'],
                      ipaddress= db['POSTGRES_ADDRESS'],
                      port= db['POSTGRES_PORT'],
                      dbname= db['POSTGRES_DBNAME'])
               )

# Create an database engine instance
connection = create_engine(postgres_str)

## Load the datasets

In [3]:
df = pd.read_sql_table('final_table', connection)
df.head()

,city,zhvi,total_pop,med_income,med_age,male,female,race_w,race_aa,race_nat,...,murder_mansl,rape,robbery,agg_assault,burglary,lar_theft,car_theft,arson,lat,long
0,Ladd,86540.17,1185.0,65972.0,42.9,587.0,598.0,1133.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,41.382535,-89.218975
1,Lake Bluff,459437.17,10003.0,107039.0,42.7,4642.0,5361.0,8616.0,151.0,62.0,...,0.0,0.0,1.0,0.0,7.0,36.0,0.0,0.0,42.278912,-87.834236
2,Lake Forest,808297.42,21131.0,171913.0,46.6,10472.0,10659.0,18890.0,520.0,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.258634,-87.840625
3,Lake Villa,208693.58,35286.0,96024.0,40.3,17646.0,17640.0,32294.0,900.0,131.0,...,1.0,4.0,0.0,4.0,3.0,29.0,1.0,0.0,42.416965,-88.073971
4,Lake Zurich,429947.17,42637.0,135146.0,42.0,21299.0,21338.0,36357.0,1008.0,145.0,...,0.0,1.0,1.0,3.0,18.0,238.0,3.0,0.0,42.196969,-88.093411


## Data preprocessing

In [4]:
clean_df = df.drop(['city','lat','long', 'male','female', 'eth_nonhisp','murder_mansl', 'robbery','arson','rape', 'car_theft','rape'],1)
clean_df.head()

,zhvi,total_pop,med_income,med_age,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,agg_assault,burglary,lar_theft
0,86540.17,1185.0,65972.0,42.9,1133.0,0.0,0.0,7.0,0.0,49.0,83.0,3.0,0.0,2.0
1,459437.17,10003.0,107039.0,42.7,8616.0,151.0,62.0,1207.0,0.0,129.0,762.0,0.0,7.0,36.0
2,808297.42,21131.0,171913.0,46.6,18890.0,520.0,94.0,1836.0,7.0,142.0,1013.0,0.0,0.0,0.0
3,208693.58,35286.0,96024.0,40.3,32294.0,900.0,131.0,1733.0,17.0,858.0,3137.0,4.0,3.0,29.0
4,429947.17,42637.0,135146.0,42.0,36357.0,1008.0,145.0,5008.0,124.0,868.0,3441.0,3.0,18.0,238.0


In [5]:
clean_df.describe()

,zhvi,total_pop,med_income,med_age,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,agg_assault,burglary,lar_theft
count,9.790000e+02,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000,979.00000,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000
mean,1.374120e+05,7628.810429,62588.514917,42.351034,6170.608401,772.398683,53.678888,422.29714,8.344211,385.104208,1054.550669,25.935649,28.830439,149.924413
std,1.178790e+05,11587.889998,21145.745980,5.858168,8906.665564,2649.138961,103.762227,1262.54303,24.359187,1965.207884,3744.702529,493.154123,380.171509,2084.391414
min,1.852242e+04,58.000000,16023.000000,21.500000,58.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.388796e+04,819.500000,49696.000000,38.650000,792.000000,1.000000,0.000000,0.00000,0.000000,0.000000,7.000000,0.000000,0.000000,0.000000
50%,9.960942e+04,1995.000000,58289.000000,42.000000,1918.000000,22.000000,9.000000,11.00000,0.000000,8.000000,39.000000,0.000000,0.000000,0.000000
75%,1.601418e+05,9741.500000,70160.000000,45.700000,7515.500000,242.500000,51.500000,121.00000,2.000000,116.000000,481.000000,4.000000,8.000000,31.500000
max,1.360530e+06,83147.000000,212750.000000,63.500000,49654.000000,30695.000000,734.000000,9528.50000,256.000000,47835.000000,74180.000000,15312.000000,11686.000000,64695.000000


In [6]:
# Split our preprocessed data into our features and target arrays
y = clean_df['zhvi']
X = clean_df.drop(['zhvi'], 1)

In [7]:
# Split and Train data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=42)

# Create a StandardScaler instances
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Linear Regression

In [8]:
# Initiate Linear regression
model = RandomForestRegressor()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

print(" MAE", mean_absolute_error(y_test,y_pred))
print(" RMSE", sqrt(mean_squared_error(y_test,y_pred)))
print(" R2", r2_score(y_test, y_pred))

 MAE 26809.564229194533
 RMSE 45926.29934694047
 R2 0.8087078935247978


In [9]:
results = pd.DataFrame({"Predicted Price": y_pred, "Actual Price": y_test}).reset_index(drop=True)


In [10]:
housing_df = pd.concat([df, results], axis=1)
housing_df.head()

,city,zhvi,total_pop,med_income,med_age,male,female,race_w,race_aa,race_nat,...,robbery,agg_assault,burglary,lar_theft,car_theft,arson,lat,long,Predicted Price,Actual Price
0,Ladd,86540.17,1185.0,65972.0,42.9,587.0,598.0,1133.0,0.0,0.0,...,0.0,3.0,0.0,2.0,0.0,0.0,41.382535,-89.218975,93975.9990,83739.50
1,Lake Bluff,459437.17,10003.0,107039.0,42.7,4642.0,5361.0,8616.0,151.0,62.0,...,1.0,0.0,7.0,36.0,0.0,0.0,42.278912,-87.834236,143344.9413,131423.58
2,Lake Forest,808297.42,21131.0,171913.0,46.6,10472.0,10659.0,18890.0,520.0,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,42.258634,-87.840625,167117.8205,162155.67
3,Lake Villa,208693.58,35286.0,96024.0,40.3,17646.0,17640.0,32294.0,900.0,131.0,...,0.0,4.0,3.0,29.0,1.0,0.0,42.416965,-88.073971,75263.9183,91324.08
4,Lake Zurich,429947.17,42637.0,135146.0,42.0,21299.0,21338.0,36357.0,1008.0,145.0,...,1.0,3.0,18.0,238.0,3.0,0.0,42.196969,-88.093411,71584.5147,81336.58


In [12]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [13]:
# Ask the customer to add a minimum and maximum temperature value
min_house_price = float(input("What is the minimum house price you would like to pay? "))
max_house_price = float(input("What is the maximum house price you would like to pay? "))


What is the minimum house price you would like to pay? 350000
What is the maximum house price you would like to pay? 450000


In [14]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = housing_df.loc[(housing_df["Actual Price"] <= max_house_price) & \
                                      (housing_df["Actual Price"] >= min_house_price)]
preferred_cities_df.head(10)

,city,zhvi,total_pop,med_income,med_age,male,female,race_w,race_aa,race_nat,...,robbery,agg_assault,burglary,lar_theft,car_theft,arson,lat,long,Predicted Price,Actual Price
90,Mark,61886.50,245.000000,77708.000000,47.700000,117.0,128.000000,245.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,41.264480,-89.250087,340475.61715,414373.75
114,Metamora,214191.25,11950.000000,88233.000000,41.800000,5873.0,6077.000000,11793.000000,97.000000,88.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,40.790592,-89.360643,368998.22280,411188.75
240,Oak Park,476229.61,17408.333333,91694.666667,41.033333,8192.0,9216.333333,12842.666667,3564.666667,154.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,41.885032,-87.784503,258970.32495,417907.42


In [15]:
city_info = preferred_cities_df[["city","total_pop",'med_age','med_income','lar_theft','Actual Price','Predicted Price', "lat", "long"]].copy()
city_info.head(10)

,city,total_pop,med_age,med_income,lar_theft,Actual Price,Predicted Price,lat,long
90,Mark,245.000000,47.700000,77708.000000,0.0,414373.75,340475.61715,41.264480,-89.250087
114,Metamora,11950.000000,41.800000,88233.000000,0.0,411188.75,368998.22280,40.790592,-89.360643
240,Oak Park,17408.333333,41.033333,91694.666667,0.0,417907.42,258970.32495,41.885032,-87.784503


In [16]:
info_box_template ="""
<dl>
<dt>city</dt><dd>{city}</dd>
<dt>total_pop</dt><dd>{total_pop}</dd>
<dt>med_income</dt><dd>{med_income}</dd>
<dt>lar_theft</dt><dd>{lar_theft}</dd>
<dt>Predicted Price</dt><dd>{Predicted Price}</dd>
</dl>
"""

# Store the DataFrame Row.
info = [info_box_template.format(**row) for index, row in city_info.iterrows()]




In [17]:
# Add a heatmap of temperature for the vacation spots.
locations = city_info[["lat", "long"]]
price = city_info["Actual Price"]
fig = gmaps.figure(center=(40.6, 89.3), zoom_level=2.5)
heat_layer = gmaps.heatmap_layer(locations, dissipating=False,
             max_intensity=100, point_radius=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))